Generación de etiquetas con label studio. Para ello primero es necesario convertir la simagenes -.tiff a .png que es compatible con label studio

In [2]:
import cv2
import os

def convert_tiff_to_png(input_dir, output_dir):
    # Check if output directory exists, create if not
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Loop through all files in the input directory
    for filename in os.listdir(input_dir):
        # Only process TIFF files
        if filename.lower().endswith('.tiff') or filename.lower().endswith('.tif'):
            input_path = os.path.join(input_dir, filename)
            
            # Read the TIFF image
            img = cv2.imread(input_path, cv2.IMREAD_UNCHANGED)
            
            if img is None:
                print(f"Error reading {filename}")
                continue

            # Construct the output filename (replace .tiff or .tif with .png)
            output_filename = os.path.splitext(filename)[0] + '.png'
            output_path = os.path.join(output_dir, output_filename)
            
            # Save the image as PNG
            cv2.imwrite(output_path, img)
            print(f"Converted {filename} to {output_filename}")

if __name__ == "__main__":
    basedir = "D:/Universidad/MsC/VA/proyecto curso/database/test"
    input_directory = basedir
    output_directory = f"{basedir}/png"
    
    convert_tiff_to_png(input_directory, output_directory)


Converted database_batch__1.tif to database_batch__1.png
Converted database_batch__14.tif to database_batch__14.png
Converted database_batch__16.tif to database_batch__16.png
Converted database_batch__17.tif to database_batch__17.png
Converted database_batch__20.tif to database_batch__20.png
Converted database_batch__4.tif to database_batch__4.png
Converted database_batch__5.tif to database_batch__5.png
Converted database_batch__7.tif to database_batch__7.png
Converted database_batch__9.tif to database_batch__9.png


Ahora vamos a importar unas anotaciones ya hechas con Label-studio y modificarlas para que encajen con otro proyecto

primero, importa las etiquetas en un archivo json y exploremos su estructura

In [2]:
import json
import os
from typing import List, Dict
import pandas as pd

# Ruta del archivo JSON 
archivo_json = "D:/Universidad/MsC/VA/proyecto curso/json labels/HR_imgs_17_20__partial_16.json"

# Función para leer y extraer información clave
def analizar_estructura(json_data: List[Dict]) -> List[Dict]:
    resumen = []
    for tarea in json_data:
        task_id = tarea.get("id")
        project = tarea.get("project")
        file_upload = tarea.get("file_upload")
        ruta_imagen = tarea.get("data", {}).get("image")
        inner_id = tarea.get("inner_id")
        for anotacion in tarea.get("annotations", []):
            annotation_id = anotacion.get("id")
            etiquetas = []
            for result in anotacion.get("result", []):
                etiqueta = {
                    "etiqueta": result["value"]["rectanglelabels"][0] if result["value"].get("rectanglelabels") else None,
                    "x": result["value"].get("x"),
                    "y": result["value"].get("y"),
                    "width": result["value"].get("width"),
                    "height": result["value"].get("height")
                }
                etiquetas.append(etiqueta)
            resumen.append({
                "task_id": task_id,
                "annotation_id": annotation_id,
                "project_id": project,
                "file_upload": file_upload,
                "ruta_imagen": ruta_imagen,
                "inner_id": inner_id,
                "etiquetas": etiquetas
            })
    return resumen

# Leer el archivo y analizar
with open(archivo_json, 'r') as f:
    datos = json.load(f)

resumen_datos = analizar_estructura(datos)

# Crear DataFrame de resumen
df_resumen = pd.json_normalize(resumen_datos, 'etiquetas', 
                               ['task_id', 'annotation_id', 'project_id', 'file_upload', 'ruta_imagen', 'inner_id'])



Ahora modificamos la parte de la estructura que nos interesa

In [3]:
import json
import copy

# Ruta del archivo original
archivo_json = "D:/Universidad/MsC/VA/proyecto curso/json labels/HR_imgs_17_20__partial_16.json"

# Leer los datos originales
with open(archivo_json, 'r') as f:
    datos = json.load(f)

# Copiamos los datos originales para modificarlos sin sobrescribir
datos_modificados = copy.deepcopy(datos)

# --- Configuración del usuario para modificación de JSON ---

# Nuevo ID de proyecto
nuevo_project_id = 8

# Mapeo de task_id originales a nuevos valores deseados
modificaciones = {
    16: {
        "nuevo_task_id": 44,
        "nuevo_annotation_id": 17,
        "nuevo_inner_id": 7,
        "nuevo_file_upload": "2b736095-database_batch_HR_16.png"
    },
    17: {
        "nuevo_task_id": 45,
        "nuevo_annotation_id": 18,
        "nuevo_inner_id": 8,
        "nuevo_file_upload": "121bfe99-database_batch_HR_17.png"
    },
    20: {
        "nuevo_task_id": 46,
        "nuevo_annotation_id": 19,
        "nuevo_inner_id": 9,
        "nuevo_file_upload": "5bda91c8-database_batch_HR_20.png"
    }
}

# Aplicar modificaciones
for tarea in datos_modificados:
    task_id_original = tarea["id"]
    if task_id_original in modificaciones:
        cambio = modificaciones[task_id_original]

        # Modificar campos principales
        tarea["id"] = cambio["nuevo_task_id"]
        tarea["inner_id"] = cambio["nuevo_inner_id"]
        tarea["project"] = nuevo_project_id
        tarea["file_upload"] = cambio["nuevo_file_upload"]
        nueva_ruta = f"/data/upload/{nuevo_project_id}/{cambio['nuevo_file_upload']}"
        tarea["data"]["image"] = nueva_ruta

        # Modificar anotaciones
        for anotacion in tarea.get("annotations", []):
            anotacion["id"] = cambio["nuevo_annotation_id"]
            anotacion["task"] = cambio["nuevo_task_id"]
            anotacion["project"] = nuevo_project_id

# Guardar resultado como nuevo archivo
archivo_salida = "D:/Universidad/MsC/VA/proyecto curso/json labels/for_project8_img_16_17_20.json"
with open(archivo_salida, 'w') as f_out:
    json.dump(datos_modificados, f_out, indent=2)

archivo_salida


'D:/Universidad/MsC/VA/proyecto curso/json labels/for_project8_img_16_17_20.json'

In [4]:
import json
import copy

# Ruta del archivo original
archivo_json = "D:/Universidad/MsC/VA/proyecto curso/json labels/imgs_16_14_danny.json"

# Leer los datos originales
with open(archivo_json, 'r') as f:
    datos = json.load(f)

# Copiamos los datos originales para modificarlos sin sobrescribir
datos_modificados = copy.deepcopy(datos)

# --- Configuración del usuario para modificación de JSON ---

# Nuevo ID de proyecto
nuevo_project_id = 8

# Mapeo de task_id originales a nuevos valores deseados
modificaciones = {
    1: {
        "nuevo_task_id": 44,
        "nuevo_annotation_id": 17,
        "nuevo_inner_id": 7,
        "nuevo_file_upload": "2b736095-database_batch_HR_16.png"
    },
    3: {
        "nuevo_task_id": 45,
        "nuevo_annotation_id": 18,
        "nuevo_inner_id": 8,
        "nuevo_file_upload": "7e1df293-database_batch_HR_14.png"
    }
}

# Aplicar modificaciones
for tarea in datos_modificados:
    task_id_original = tarea["id"]
    if task_id_original in modificaciones:
        cambio = modificaciones[task_id_original]

        # Modificar campos principales
        tarea["id"] = cambio["nuevo_task_id"]
        tarea["inner_id"] = cambio["nuevo_inner_id"]
        tarea["project"] = nuevo_project_id
        tarea["file_upload"] = cambio["nuevo_file_upload"]
        nueva_ruta = f"/data/upload/{nuevo_project_id}/{cambio['nuevo_file_upload']}"
        tarea["data"]["image"] = nueva_ruta

        # Modificar anotaciones
        for anotacion in tarea.get("annotations", []):
            anotacion["id"] = cambio["nuevo_annotation_id"]
            anotacion["task"] = cambio["nuevo_task_id"]
            anotacion["project"] = nuevo_project_id

# Guardar resultado como nuevo archivo
archivo_salida = "D:/Universidad/MsC/VA/proyecto curso/json labels/for_project8_img_16_14.json"
with open(archivo_salida, 'w') as f_out:
    json.dump(datos_modificados, f_out, indent=2)

archivo_salida


'D:/Universidad/MsC/VA/proyecto curso/json labels/for_project8_img_16_14.json'